In [1]:
from pyspark.sql.functions import col
from pyspark.sql import SQLContext, SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from collections import namedtuple
from pyspark.ml import PipelineModel
import pandas as pd
from pyspark.sql.functions import when


sc = SparkContext("local[2]", "Streaming App")
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

socket_stream = ssc.socketTextStream("localhost", 5559) 

lines = socket_stream.window(20)
fields = ("SentimentText")
Tweet = namedtuple( 'Tweet', fields )
termination_flag = 0
rf = PipelineModel.load("file://///Users/zaranabhalala/Desktop/project/logreg.model")

def getSparkSessionInstance(sparkConf):
    if ("sparkSessionSingletonInstance" not in globals()):
        globals()["sparkSessionSingletonInstance"] = SparkSession \
            .builder \
            .config(conf=sparkConf) \
            .getOrCreate()
    return globals()["sparkSessionSingletonInstance"]

def rdd_iterator(time, rdd):
        
        print("========= %s =========" % str(time))
        
        print("Started")
        spark = getSparkSessionInstance(rdd.context.getConf())
        print("spark session")
        rowRdd = rdd.map(lambda w: Tweet(w))
        linesDataFrame = spark.createDataFrame(rowRdd)
        print("linesDataframe created.")
        linesDataFrame.createOrReplaceTempView("tweets")
    
        lineCountsDataFrame = spark.sql("select SentimentText as text from tweets")
        print(lineCountsDataFrame.show(5, False))
        
        prediction = rf.transform(lineCountsDataFrame)
        print("after prediction")
        keep_list = ["text", "prediction"]
        prediction_save = prediction.select([column for column in prediction.columns if column in keep_list])

        prediction_save.show(50, False)
        pred = prediction_save.toPandas()
        pred.to_csv('preds.csv', mode='a', header='true', index = False, encoding='utf-8')
        import time
        time.sleep(120)
        ssc.stop()


lines.foreachRDD(rdd_iterator)

ssc.start()
ssc.awaitTermination()


/Users/zaranabhalala/opt/anaconda3/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


========= 2021-12-20 19:49:20 =========
Started
spark session
linesDataframe created.
+---------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                       |
+---------------------------------------------------------------------------------------------------------------------------+
|RT tsumurajp  Amazon tsumurajp RT                                                                                          |
|RT ahamoofficial DayAmazon ahamo ahamoofficial                                                                             |
|My nigga randomly cleared out my Amazon cart I see some neck under the tree along with his Christmas gifts                 |
|RT eiking   Amazon or PayPay RTRT                                                                                          |
|RT TudorTweep New ReleaseLady o